In [25]:
import SimpleITK as sitk
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader

In [2]:
path = '/home/ikboljon.sobirov/data/fs1_research/Ikboljon.Sobirov/imagecas/imagecas/resampled_space/'

patients = os.listdir(path)
len(patients)


200

In [67]:
class CasDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        self.patients = os.listdir(self.path)

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient = self.patients[idx]
        img = self.read_image(os.path.join(self.path, patient, (patient+'_img.nii.gz')))
        mask = self.read_image(os.path.join(self.path, patient, (patient+'_seg.nii.gz')))
        img = np.expand_dims(img, axis=0)
        mask = np.expand_dims(mask, axis=0)

        if self.transform:
            img = self.transform(img)
            # mask = self.transform(mask)
            
        
        
        return img, mask
    
    @staticmethod
    def read_image(path):
        img = sitk.ReadImage(path)
        img = sitk.GetArrayFromImage(img)
        return img
    

In [68]:
from torchvision import transforms
import scipy.ndimage

class Resize3DTransform:
    def __init__(self, size):
        self.size = size

    def __call__(self, volume):
        assert len(volume.shape) == len(self.size), "Input volume and size dimensions do not match"
        resized_volume = scipy.ndimage.zoom(volume, self.size, order=3)
        return resized_volume

class ClipZeroOneTransform:
    def __call__(self, image):
        image = np.clip(image, 0, 1)
        return image


In [69]:
from torchvision import transforms


# Apply transforms to the dataset
transform = transforms.Compose([
    # Resize3DTransform(size=(256, 256, 256)),
    ClipZeroOneTransform()
])  

# Apply transforms to the dataset
# Create an instance of the CasDataset
dataset = CasDataset(path, transform=transform)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [70]:
sample = next(iter(dataloader))

In [71]:
sample[0].shape, sample[1].shape

(torch.Size([1, 1, 138, 163, 163]), torch.Size([1, 1, 138, 163, 163]))

In [66]:
for batch in dataloader:
    img, mask = batch
    print(img.shape, mask.shape)
    break

torch.Size([1, 1, 138, 176, 176]) torch.Size([1, 1, 138, 176, 176])
